In [1]:
#Importing necessary libraries
import pandas as pd 
import numpy as np
import time

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException,WebDriverException,ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException,ElementNotInteractableException,NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings 
warnings.filterwarnings("ignore")

# Scraping data from www.yatra.com

In [8]:
airline=[]
flight_id=[]
date=[]
departure_time=[]
arrival_time=[]
from_city=[]
to_city=[]
duration=[]
stops=[]
price=[]

dep_city="New Delhi"

def city_name(city_code):
    if city_code=='BOM':
        return 'Mumbai'
    elif city_code=='BLR':
        return 'Bangalore'
    elif city_code=='CCU':
        return 'Kolkata'
    elif city_code=='HYD':
        return 'Hyderabad'
    else:
        return 'Not Defined'

def get_data(flight_data,day,arrival_city):
    try:
        components= i.text.strip().split("\n")
        airline.append(components[0])
        flight_id.append(components[1])
        date.append(f'{day} Dec')
        departure_time.append(components[2])
        arrival_time.append(components[4])
        from_city.append(dep_city)
        to_city.append(arrival_city)
        duration.append(components[-4])
        stops.append(components[-3])
        price.append(components[-2])
    except AttributeError:
        pass

In [16]:
driver= webdriver.Chrome("chromedriver.exe")
month='12'
city_codes=['BOM','BLR','CCU','HYD']
for city in city_codes:
    arrival_city=city_name(city)
    for day in range(16,32):
        link= f'https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination={city}&destinationCountry=IN&flexi=0&flight_depart_date={day}%2F{month}%2F2022&hb=0&noOfSegments=1&origin=DEL&originCountry=IN&type=O&unique=418977921272&version=1.25&viewName=normal'
        driver.get(link)
        try:
            wait= WebDriverWait(driver,40).until(EC.presence_of_element_located((By.XPATH,"//div[@class='flight-det table full-width clearfix']")))
        except TimeoutException:
            driver.close()
            time.sleep(2)
            driver= webdriver.Chrome("chromedriver.exe")
            driver.get(link)
            wait= WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,"//div[@class='flight-det table full-width clearfix']")))
        time.sleep(1)
        html= driver.find_element(By.TAG_NAME,'html')
        for i in range(0,50):    
            html.send_keys(Keys.PAGE_DOWN)
            time.sleep(1)
        data= driver.find_elements(By.XPATH,"//div[@class='flight-det table full-width clearfix']")
        for i in data:
            get_data(i,day,arrival_city)
driver.close()

In [17]:
results_df= pd.DataFrame({'Airline':airline,
                          'Flight_ID':flight_id,
                          'Date':date,
                          'Departure Time':departure_time,
                          'Arrival Time':arrival_time,
                          'Source':from_city,
                          'Destination':to_city,
                          'Duration':duration,
                          'Stops':stops,
                          'Price':price})
results_df

,Airline,Flight_ID,Date,Departure Time,Arrival Time,Source,Destination,Duration,Stops,Price
0,Go First,G8-530,16 Dec,07:00,09:10,New Delhi,Mumbai,2h 10m,Non Stop,"5,953"
1,Go First,G8-334,16 Dec,08:00,10:10,New Delhi,Mumbai,2h 10m,Non Stop,"5,953"
2,Go First,G8-336,16 Dec,14:20,16:35,New Delhi,Mumbai,2h 15m,Non Stop,"5,953"
3,Go First,G8-346,16 Dec,19:40,21:55,New Delhi,Mumbai,2h 15m,Non Stop,"5,953"
4,Go First,G8-330,16 Dec,21:00,23:15,New Delhi,Mumbai,2h 15m,Non Stop,"5,953"
...,...,...,...,...,...,...,...,...,...,...
9153,SpiceJet,SG-8171/4002,31 Dec,12:00,09:40,New Delhi,Hyderabad,21h 40m,1 Stop,"19,500"
9154,Air Asia,I5-1529/1983,31 Dec,09:35,16:50,New Delhi,Hyderabad,7h 15m,1 Stop,"20,889"
9155,Air Asia,I5-1529/972,31 Dec,09:35,17:20,New Delhi,Hyderabad,7h 45m,1 Stop,"20,889"
9156,Air Asia,I5-798/1518,31 Dec,12:45,20:25,New Delhi,Hyderabad,7h 40m,1 Stop,"24,356"


In [18]:
x=results_df.shape[0]-1
start_date= results_df['Date'][0]
end_date=results_df['Date'][x]
results_df.to_csv(f'Results_{start_date}-{end_date}.csv')

# Creating the final dataset

In [2]:
df1= pd.read_csv('Results_22 Sep-24 Sep.csv')
df2= pd.read_csv('Results_25 Sep-30 Sep.csv')
df3= pd.read_csv('Results_1 Oct-15 Oct.csv')
df4= pd.read_csv('Results_16 Oct-23 Oct.csv')
df5= pd.read_csv('Results_24 Oct-31 Oct.csv')
df6= pd.read_csv('Results_1 Nov-15 Nov.csv')
df7= pd.read_csv('Results_16 Nov-30 Nov.csv')
df8= pd.read_csv('Results_1 Dec-15 Dec.csv')
df9= pd.read_csv('Results_16 Dec-31 Dec.csv')

In [3]:
data= [df1,df2,df3,df4,df5,df6,df7,df8,df9]
df= pd.concat(data,axis=0)
df.reset_index(inplace=True)
df

,index,Unnamed: 0,Airline,Flight_ID,Date,Departure Time,Arrival Time,Source,Destination,Duration,Stops,Price
0,0,0,Go First,G8-334,22 Sep,09:50,12:00,New Delhi,Mumbai,2h 10m,Non Stop,"5,950"
1,1,1,Go First,G8-336,22 Sep,14:30,16:40,New Delhi,Mumbai,2h 10m,Non Stop,"5,950"
2,2,2,SpiceJet,SG-8709,22 Sep,19:00,21:10,New Delhi,Mumbai,2h 10m,Non Stop,"5,950"
3,3,3,Go First,G8-330,22 Sep,21:05,23:15,New Delhi,Mumbai,2h 10m,Non Stop,"5,950"
4,4,4,SpiceJet,SG-8701,22 Sep,07:20,09:35,New Delhi,Mumbai,2h 15m,Non Stop,"5,950"
...,...,...,...,...,...,...,...,...,...,...,...,...
59702,9153,9153,SpiceJet,SG-8171/4002,31 Dec,12:00,09:40,New Delhi,Hyderabad,21h 40m,1 Stop,"19,500"
59703,9154,9154,Air Asia,I5-1529/1983,31 Dec,09:35,16:50,New Delhi,Hyderabad,7h 15m,1 Stop,"20,889"
59704,9155,9155,Air Asia,I5-1529/972,31 Dec,09:35,17:20,New Delhi,Hyderabad,7h 45m,1 Stop,"20,889"
59705,9156,9156,Air Asia,I5-798/1518,31 Dec,12:45,20:25,New Delhi,Hyderabad,7h 40m,1 Stop,"24,356"


In [4]:
df.drop(columns=['index','Unnamed: 0'],inplace=True)

In [5]:
df['Price'][15:20]

15             5,954
16             5,954
17             5,954
18    1 left at5,955
19    1 left at5,955
Name: Price, dtype: object

In [6]:
price_list=[]
ticket_status=[]
for i in range(len(df['Price'])):
    value=df['Price'][i].replace(',','')
    status_check= df['Price'][i].find('left at')
    if status_check!=(-1):
        value=value.replace('left at','')
        values= value.split(" ")
        price_list.append(values[-1])
        ticket_status.append(f'{values[0]} left')
    else:
        price_list.append(value)
        ticket_status.append('Available')

price_list[15:20]

['5954', '5954', '5954', '5955', '5955']

In [7]:
df['Ticket Status']=ticket_status

In [8]:
df['Price']=pd.to_numeric(price_list)

In [10]:
df= df.sample(frac=1)
df

,Airline,Flight_ID,Date,Departure Time,Arrival Time,Source,Destination,Duration,Stops,Price,Ticket Status
10752,Go First,G8-336/398,14 Oct,14:30,22:25,New Delhi,Bangalore,7h 55m,1 Stop,8155,Available
28758,Go First,G8-323/325,14 Nov,18:05,00:35,New Delhi,Bangalore,6h 30m,1 Stop,8155,Available
49577,Air India,AI-665/615,8 Dec,08:00,07:30,New Delhi,Hyderabad,23h 30m,1 Stop,9840,1 left
16214,Go First,G8-286,16 Oct,10:40,15:00,New Delhi,Bangalore,4h 20m,1 Stop,10708,1 left
36538,Air Asia,I5-711/1453,23 Nov,06:05,12:30,New Delhi,Bangalore,6h 25m,1 Stop,7425,Available
...,...,...,...,...,...,...,...,...,...,...,...
2309,Air India,AI-403/656,29 Sep,12:50,19:15,New Delhi,Mumbai,6h 25m,1 Stop,10017,Available
19503,IndiGo,6E-5042/5207,24 Oct,10:15,17:25,New Delhi,Mumbai,7h 10m,1 Stop,7103,Available
39683,Go First,G8-171/394,30 Nov,18:30,04:00,New Delhi,Kolkata,9h 30m,1 Stop,8578,Available
22943,Vistara,UK-829/878/773,30 Oct,07:05,19:55,New Delhi,Kolkata,12h 50m,2 Stop(s),17198,Available


In [11]:
df.reset_index(inplace=True,drop=True)
df

,Airline,Flight_ID,Date,Departure Time,Arrival Time,Source,Destination,Duration,Stops,Price,Ticket Status
0,Go First,G8-336/398,14 Oct,14:30,22:25,New Delhi,Bangalore,7h 55m,1 Stop,8155,Available
1,Go First,G8-323/325,14 Nov,18:05,00:35,New Delhi,Bangalore,6h 30m,1 Stop,8155,Available
2,Air India,AI-665/615,8 Dec,08:00,07:30,New Delhi,Hyderabad,23h 30m,1 Stop,9840,1 left
3,Go First,G8-286,16 Oct,10:40,15:00,New Delhi,Bangalore,4h 20m,1 Stop,10708,1 left
4,Air Asia,I5-711/1453,23 Nov,06:05,12:30,New Delhi,Bangalore,6h 25m,1 Stop,7425,Available
...,...,...,...,...,...,...,...,...,...,...,...
59702,Air India,AI-403/656,29 Sep,12:50,19:15,New Delhi,Mumbai,6h 25m,1 Stop,10017,Available
59703,IndiGo,6E-5042/5207,24 Oct,10:15,17:25,New Delhi,Mumbai,7h 10m,1 Stop,7103,Available
59704,Go First,G8-171/394,30 Nov,18:30,04:00,New Delhi,Kolkata,9h 30m,1 Stop,8578,Available
59705,Vistara,UK-829/878/773,30 Oct,07:05,19:55,New Delhi,Kolkata,12h 50m,2 Stop(s),17198,Available


In [12]:
df.to_excel('Indian_Flights_Dataset.xlsx')